In [23]:
from pymatgen.io.openmm.generators import OpenMMSolutionGen
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Generators

A pymatgen InputSet contains all of the files needed to set up and run a simulation. Writing those files by hand would be incredibly tedious, so we use a Generator to create the InputSet. The Generator is instantiated with simulation properties as input, like temperature and force field. Once it is created, it can generate many InputSets with different sets of molecules.

In [31]:
generator = OpenMMSolutionGen(
    default_force_field="sage",
    temperature= 298,
    step_size= 0.001,
    default_charge_method= "mmff94",
)

### Setting parameterizer arguments when getting an input set

An InputSet is created by calling `generator.get_input_set()` and providing dicts defining each molecule and the density of the solution. In addition to the required parameters there several optional parameters. This included parameters which allow you to define how the system is parameterized, what force fields should be used, any custom force field files to leverage and more. Below are more detailed descriptions of each of the parameters and examples of using them

#### parameterizer_type
The `parameterizer_type` argument specifies the type of tooling use for parameterization. It can be set to one of the ParameterizerType options (`INTERCHANGE`, `OPENMM`, `DEFAULT`). Note that the `DEFAULT` ParameterizerType which is the parameterizer used by Default, simply maps to the `OPENMM` parameterizer configuration. Another thing to note is that in order for this value to be used you must also set the `parameterizer_assignment` parameter to be `EXPLICIT` otherwise the type will be assigned based on the force fields you specify. 

#### parameterizer_assignment
The `parameterizer_assignment` argument specifies how the parameterizer should be assigned for building the system. It can be set to one of the ParameterizerAssignment options (`INFERRED` or `EXPLICIT`). 


Here's an example of how use these parameters when generating an input set: 

In [32]:
from pymatgen.io.openmm.parameterizer import (
    ParameterizerType,
    ParameterizerAssignment
)

# building mol dict for an ionic system which uses the amoeba force field
input_mol_dicts = [
    {
      "smile": "O",
      "force_field": "amoeba",
      "count": 200
    },
    {
      "smile":"[Cl-]",
      "force_field": "amoeba",
      "count": 50,
      "default_charge_method": "mmff94"
    }
]


input_set = generator.get_input_set(
    input_mol_dicts,
    parameterizer_type=ParameterizerType.OPENMM_PARAMETERIZER,
    parameterizer_assignment=ParameterizerAssignment.EXPLICIT,
    density=1
)

in tempfile


We could also generate the same input set leveraging the defaults since the default parameterizer for "amoeba" is `OPENMM_PARAMETERIZER` and the parameterizer is implicitly assigned by default

In [33]:
# building mol dict for an ionic system which uses the amoeba force field
input_mol_dicts = [
    {
      "smile": "O",
      "force_field": "amoeba",
      "count": 200
    },
    {
      "smile":"[Cl-]",
      "force_field": "amoeba",
      "count": 50,
      "default_charge_method": "mmff94"
    }
]


input_set = generator.get_input_set(
    input_mol_dicts,
    density=1
)

in tempfile


Note: Not all combinations of force_field, parameterizer_type, and parameterizer_assignment will be valid. The good news is there should be some clear error messages about what is and is not supported. For example if you try to create a molocule with a forcfield only supported by the `OPENMM_PARAMETERIZER` but you set the `parameterizer_type` to `INTERCHANGE_PARAMETERIZER` and `parameterizer_assignement` to `EXPLICIT` you will get a message indicating that the parameterizer you are using does not support the force field type. This is demonstrated below:

In [ ]:
from pymatgen.io.openmm.parameterizer import (
    ParameterizerType,
    ParameterizerAssignment
)

# building mol dict for an ionic system which uses the amoeba force field
input_mol_dicts = [
    {
      "smile": "O",
      "force_field": "sage",
      "count": 200
    },
    {
      "smile":"[Cl-]",
      "force_field": "sage",
      "count": 50,
      "default_charge_method": "mmff94"
    }
]


input_set = generator.get_input_set(
    input_mol_dicts,
    parameterizer_type=ParameterizerType.OPENMM_PARAMETERIZER,
    parameterizer_assignment=ParameterizerAssignment.EXPLICIT,
    density=1
)

### customize_force_field and custom_file_paths
The `customize_force_field` argument specifies whether or not to use custom force field files for parameterization, while the `custom_file_paths argument` specifies a list of file paths to custom force field files. These are set to `false` and `None` respectively by default Here's an example of how to use these arguments:

```python

input_set = generator.get_input_set(
    input_mol_dicts,
    parameterizer_type=ParameterizerType.OPENMM_PARAMETERIZER,
    parameterizer_assignment=ParameterizerAssignment.EXPLICIT,
    customize_force_field: true,
    custom_file_paths: ["path/to/custom/forcefield.xml"],
    density=1
)

```